In [1]:
from captum.attr import IntegratedGradients
from captum.attr import visualization as viz
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torch.nn.functional as F

from Pneumonia_predictor import PneumoniaPredictorCNN, PneumoniaDataset, PneumoniaPredictorCNN_gradcam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import yaml

import cv2

import argparse
import os
import cv2
import numpy as np
import torch
from torchvision import models
from pytorch_grad_cam import (
    GradCAM, FEM, HiResCAM, ScoreCAM, GradCAMPlusPlus,
    AblationCAM, XGradCAM, EigenCAM, EigenGradCAM,
    LayerCAM, FullGrad, GradCAMElementWise, KPCA_CAM, ShapleyCAM,
    FinerCAM
)
from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import (
    show_cam_on_image, deprocess_image, preprocess_image
)
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget, ClassifierOutputReST



In [2]:

model_name = '6_random_labels_bs32_lr0.0001_epoch15_img_size224x224'
batch_size = 1

output_dir = Path('output/model/')
model_path = Path(output_dir/f'{model_name}_final.pth')
model_config = Path(output_dir/f'{model_name}_config.yaml')

with open(model_config, 'r') as f:
    config = yaml.safe_load(f)

seed = config['seed']
torch.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu")


model_params  = {'image_size':config['image_size'],
                'in_channels':config['in_channels'], 
                'conv_defs':config['conv_layers'], 
                'fc_defs':config['fc_layers'],
                'fc_dropout':config['fc_dropout'],
                'fc_batch_norm':config['fc_batch_norm']
                }


model =PneumoniaPredictorCNN(**model_params)
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.to(device)

model.eval()



PneumoniaPredictorCNN(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), pad

In [3]:
# laod test data and show  some images

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

test_data_df = pd.read_csv(f'data/test_data.csv')

test_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_dataset = PneumoniaDataset(test_data_df, transform=test_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
dataiter = iter(test_loader)



map_labels = {0: 'Normal', 1: 'Pneumonia'}


In [4]:
# i want to get the file name of the image from 
test_dataset = PneumoniaDataset(test_data_df, transform=test_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
dataiter = iter(test_loader)

In [7]:
for batch in test_loader:
    target_layers = [model.conv[31]]

    image_tensor, label, file_path_tuple = batch # Renamed 'image' to 'image_tensor' for clarity
    image_tensor = image_tensor.to(device)

    targets = [ClassifierOutputTarget(0)]

    outputs = model(image_tensor)           # (B,1) raw logits
    logits  = outputs.squeeze(1)      # -> (B,)

    # apply sigmoid & threshold at 0.5:
    probs   = torch.sigmoid(logits)   # (B,) in [0,1]
    preds   = (probs > 0.5).long()    # (B,) in {0,1}

    cam_algorithm = GradCAM
    with cam_algorithm(target_layers=target_layers,
                    model=model,) as cam:

        grayscale_cam = cam(input_tensor=image_tensor, targets=targets)
        grayscale_cam = grayscale_cam[0, :]

        # Prepare original image for overlay.
        # image_tensor is (B, C, H, W), e.g., (1, 1, 224, 224)
        # Convert to (H, W, C) numpy array, normalized to 0-1 float for show_cam_on_image
        image_for_overlay = image_tensor[0].cpu().numpy()  # shape: (C, H, W)
        image_for_overlay = np.transpose(image_for_overlay, (1, 2, 0))  # shape: (H, W, C)

        # denormalize the image
        image_for_overlay = image_for_overlay * 0.5 + 0.5  # unnormalize

        if image_for_overlay.shape[2] == 1:
            img_for_overlay = np.repeat(image_for_overlay, 3, axis=2) # (H, W, 3)

        cam_image = show_cam_on_image(image_for_overlay, grayscale_cam, use_rgb=True)

        cam_image = cv2.cvtColor(cam_image, cv2.COLOR_RGB2BGR) # Convert back to BGR for cv2.imwrite


    gb_model = GuidedBackpropReLUModel(model=model, device=device)
    gb = gb_model(image_tensor, target_category=0)

    gb_numpy_hwc = gb.transpose( 0, 1, 2)  # shape: (H, W, C)

    gb_output_visual = deprocess_image(gb_numpy_hwc)

    # create cam_gb 
    cam_mask_rgb = cv2.merge([grayscale_cam, grayscale_cam, grayscale_cam])

    # 2. Multiply with gb_numpy_hwc.
    #    gb_numpy_hwc is (H, W, C_in). If C_in is 1, it broadcasts: (H,W,3) * (H,W,1) -> (H,W,3)
    #    The values in gb_numpy_hwc are raw gradients.
    product_cam_gb = cam_mask_rgb * gb_numpy_hwc # (H, W, 3)


    cam_gb_visual = deprocess_image(product_cam_gb)

    out_dir = 'output/gradcam_layer31_random_labels/'
    os.makedirs(out_dir, exist_ok=True)

    file_basename = os.path.basename(file_path_tuple[0])
    file_name = os.path.splitext(file_basename)[0]
    file_name = f'{file_name}_pred_{preds[0].item()}_prob_{probs[0].item():.3f}'

    cam_output_path = os.path.join(out_dir, f'{file_name}_cam.jpg')
    gb_output_path = os.path.join(out_dir, f'{file_name}_gb.jpg')
    cam_gb_output_path = os.path.join(out_dir, f'{file_name}_cam_gb.jpg')

    cv2.imwrite(cam_output_path, cam_image)
    # cv2.imwrite(gb_output_path, gb_output_visual)     # Should now be (H,W,1) or (H,W,3) uint8
    # cv2.imwrite(cam_gb_output_path, cam_gb_visual) # Should now be (H,W,3) uint8

In [6]:
with torch.no_grad():
    images = norm_images
    # make labels floats if you didn't already:
    labels = norm_labels

    outputs = model(images)           # (B,1) raw logits
    logits  = outputs.squeeze(1)      # -> (B,)

    # apply sigmoid & threshold at 0.5:
    probs   = torch.sigmoid(logits)   # (B,) in [0,1]
    preds   = (probs > 0.5).long()    # (B,) in {0,1}
    print(f'Normal: predictions: {preds} | probabilities: {[round(p, 3) for p in probs.tolist()]}')

    images = pneu_images
    labels = pneu_labels
    outputs = model(images)           # (B,1) raw logits
    logits  = outputs.squeeze(1)      # -> (B,)

    probs  = torch.sigmoid(logits)   # (B,) in [0,1]
    preds  = (probs > 0.5).long()    # (B,) in {0,1}
    print(f'Pneumonia: predictions: {preds} | probabilities: {[round(p, 3) for p in probs.tolist()]}')


NameError: name 'norm_images' is not defined